<a href="https://colab.research.google.com/github/AhlemKaabi/holbertonschool-machine_learning/blob/main/supervised_learning/0x12-transformer_apps/Transformer_Applications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
mygenerator = (x*x for x in range(3))

In [ ]:
mygenerator

<generator object <genexpr> at 0x7fcbeaf409d0>

In [ ]:
""" Transformer Applications - 1-2-3-Encode Tokens """

class Dataset():
    """ Dataset loads and preps a dataset for machine translation """

    def __init__(self, batch_size, max_len):
        """
        Method:
        -------
            Class constructor

        Parameters:
        -----------
            batch_size: the batch size for training/validation.
            max_len: the maximum number of tokens allowed per example sentence.
        """
        # https://youtu.be/YrMy-BAqk8k
        (data_train, data_valid), ds_info  = tfds.load(
            "ted_hrlr_translate/pt_to_en",
            split=["train", "validation"],
            as_supervised=True,
            with_info=True
        )
        # as_supervised=True: return tuple(img, lable), False return dict.
        # we can add with_info=True

        tokenizer_pt, tokenizer_en = self.tokenize_dataset(data_train)
        self.tokenizer_pt = tokenizer_pt
        self.tokenizer_en = tokenizer_en

        # update the data_train and data_validate attributes
        # by tokenizing the examples
        self.data_train = data_train.map(self.tf_encode)
        self.data_valid = data_valid.map(self.tf_encode)

        # UPDATE data_train
        # 1- filter out all examples that have either sentence with more than
        # max_len tokens
        # https://www.tensorflow.org/api_docs/python/tf/data/Dataset#filter
        # https://www.tensorflow.org/api_docs/python/tf/math/logical_and
        self.data_train = self.data_train.filter(
            lambda pt, en: tf.math.logical_and(tf.size(pt) <= max_len,
                                               tf.size(en) <= max_len))

        # 2- cache the dataset to increase performance
        # https://www.tensorflow.org/api_docs/python/tf/data/Dataset#cache
        self.data_train = self.data_train.cache()
        # 3- shuffle the entire dataset
        # https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle
        buffer_size =ds_info. splits['train'].num_examples
        self.data_train = self.data_train.shuffle(buffer_size)
        # 4- split the dataset into padded batches of size batch_size
        # https://www.tensorflow.org/api_docs/python/tf/data/Dataset#padded_batch
        self.data_train = self.data_train.padded_batch(batch_size)
        # 5- prefetch the dataset using tf.data.experimental.AUTOTUNE to
        # increase performance
        # https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch
        # https://www.tensorflow.org/api_docs/python/tf/data/experimental#other-members_1
        self.data_train = self.data_train.prefetch(
            tf.data.experimental.AUTOTUNE
            )
        # UPDATE data_valid
        # filter out all examples that have either sentence with more than
        # max_len tokens
        self.data_valid = self.data_valid.filter(
            lambda pt, en: tf.math.logical_and(tf.size(pt) <= max_len,
                                               tf.size(en) <= max_len))
        # split the dataset into padded batches of size batch_size
        self.data_valid = self.data_valid.padded_batch(batch_size)


    def tokenize_dataset(self, data):
        """
        Method:
        -------
            Creates sub-word tokenizers for our dataset

        Parameters:
        -----------
            data (tuple): formatted as (pt, en)
                pt: the tf.Tensor containing the Portuguese sentence
                en: the tf.Tensor containing the corresponding English
                sentence

        Returns:
        --------
            tokenizer_pt: the Portuguese tokenizer
            tokenizer_en: the English tokenizer
        """
        text_Encoder = tfds.deprecated.text.subword_text_encoder.SubwordTextEncoder
        # then use build_from_corpus()
        # https://www.tensorflow.org/datasets/api_docs/python/tfds/deprecated/text/SubwordTextEncoder#build_from_corpus

        # generator yielding str, from which subwords will be constructed.
        # what is generator? ->
        # https://stackoverflow.com/questions/59576060/subwordtextencoder-build-from-corpus-splits-tokens-in-reserved-tokens
        # https://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do

          # trying: corpus_generator_en = (en for _, en in data_train)

          # but: TypeError -> Expected binary or unicode string,
          # got <tf.Tensor: shape=(), dtype=string, numpy=b'and when you improve
        # searchability , you actually take away the one advantage of print ,
         # which is serendipity .'>

        # corpus_generator_en = (en.as_numpy() for _, en in data_train)
        # corpus_generator_pt = (pt.as_numpy_iterator() for pt,_ in data_train)
        # error:
        # 'tensorflow.python.framework.ops.EagerTensor' object
        # has no attribute 'as_numpy'
        # solution -> useful function, numpy!!!!! already we have a binary
        # string ;)
        # https://www.tensorflow.org/api_docs/python/tf/Tensor#expandable-1
        corpus_generator_en = (en.numpy() for _, en in data)
        corpus_generator_pt = (pt.numpy() for pt, _ in data)

        target_vocab_size = 2**15
        tokenizer_en = text_Encoder.build_from_corpus(
            corpus_generator_en,
            target_vocab_size
        )

        tokenizer_pt = text_Encoder.build_from_corpus(
            corpus_generator_pt,
            target_vocab_size
        )
        return tokenizer_pt, tokenizer_en

    def encode(self, pt, en):
        """
        Method:
        -------
            Encodes a translation into tokens.(encode sub-word into tokens)

        Parameters:
        -----------
            pt (tf.Tensor):  the Portuguese sentence.
            en (tf.Tensor): the corresponding English sentence.

        Returns:
        --------
            tokenizer_pt (np.ndarray): the Portuguese tokenizer
            tokenizer_en (np.ndarray): the English tokenizer

        ** The tokenized sentences should include the start and **
                        end of sentence tokens

        ** The start token should be indexed as vocab_size **

        ** The end token should be indexed as vocab_size + 1 **
        """
        en = self.tokenizer_en.encode(en.numpy())
        vocab_size_en = self.tokenizer_en.vocab_size
        en.insert(0, vocab_size_en)
        en.append(vocab_size_en + 1)

        pt = self.tokenizer_pt.encode(pt.numpy())
        vocab_size_pt = self.tokenizer_pt.vocab_size
        pt.insert(0, vocab_size_pt)
        pt.append(vocab_size_pt + 1)

        return pt, en

    def tf_encode(self, pt, en):
        """
        Method:
        -------
            Acts as a tensorflow wrapper for the encode instance method.

        Parameters:
        -----------
            pt (tf.Tensor):  the Portuguese sentence
            en (tf.Tensor): the corresponding English sentence

        Returns:
        --------
            tensors

        ** Make sure to set the shape of the pt and en return tensors **
        """
        pt, en = tf.function(func=self.encode,input_signature=[pt, en],Tout=tf.int64)
        return pt, en

In [ ]:
""" Transformer Applications - 4-Create Masks """

def create_masks(inputs, target):
    """
    Method:
    -------
        Creates all masks for training/validation.

    Parameters:
    -----------
        inputs (tf.Tensor of shape (batch_size, seq_len_in)):
        contains the input sentence.

        target (tf.Tensor of shape (batch_size, seq_len_out)):
        contains the target sentence

    Returns:
        encoder_mask (tf.Tensor of shape (batch_size, 1, 1, seq_len_in)):
        `padding mask` to be applied in the encoder.

        combined_mask (tf.Tensor of shape (batch_size,
                                           1,
                                           seq_len_out,
                                           seq_len_out):
        used in the 1st attention block in the decoder to pad and mask future
        tokens in the input received by the decoder. It takes the maximum
        between a look ahead mask and the decoder target padding mask.

        decoder_mask (tf.Tensor  of shape (batch_size, 1, 1, seq_len_in)):
        `padding mask` used in the 2nd attention block in the decoder.


    **  This function should only use tensorflow operations in order to **
    **            properly function in the training step                **

    """
    def create_padding_mask(seq):
        """ """
        seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

        # add extra dimensions to add the padding to the attention logits.
        return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

    def create_look_ahead_mask(size):
        """ """
        # tf.linalg.bandPart (a, numLower, numUpper)
        # Parameters:
        # a: it is tf.tensor to be passed.
        # numLower: Number of subdiagonals to keep.
        #           If negative, keep entire lower triangle.
        # numUpper: Number of subdiagonals to keep.
        #           If negative, keep entire upper triangle.
        mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
        return mask  # (seq_len, seq_len)


    batch_size, seq_len_out = target.shape

    encoder_mask = create_padding_mask(inputs)

    dec_target_padding_mask = create_padding_mask(target)
    look_ahead_mask = create_look_ahead_mask(seq_len_out)

    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    decoder_mask = create_padding_mask(inputs)

    return encoder_mask, combined_mask, decoder_mask


In [ ]:
""" main 4 """
tf.compat.v1.set_random_seed(0)
data = Dataset(32, 40)

TypeError: ignored

In [ ]:
for inputs, target in data.data_train.take(1):
    print(create_masks(inputs, target))

TypeError: ignored